# Search index

Using OpenSearch to create index mappings for the recipes

## Import and parse JSON file with the recipes

In [1]:
import json as json

with open("recipes_data.json", "r") as read_file:
    recipes_data = json.load(read_file)

In [2]:
j = len(recipes_data)
str(j)

'994'

## OpenSearch client creation


Need to create the openSearch client in order to create the search index mappings and further quering

### Credentials and index name

In [3]:
import pprint as pp
import requests
from config import CONFIG

host = CONFIG["host"]
port = CONFIG["port"]
user = CONFIG["user"]
password = CONFIG["password"]
#index_name = CONFIG["index_name"]
index_name = user

print(index_name)

user219


### Create the OpenSearch client and check if index_name exists

In [4]:
from opensearchpy import OpenSearch
from opensearchpy import helpers

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

# index_name_test = 'python-test-index'
# index_body_test = {
#   'settings': {
#     'index': {
#       'number_of_shards': 4
#     }
#   }
# }

# response = client.indices.create(index_name_test, body=index_body_test)

## Index mappings

In [5]:
# recipe_mapping = {
#     "properties": {
#             "test": {"type": "keyword"},
#         }
# }

### Print index properties

In [ ]:
print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
index_settings = {
    "settings":{
      "index":{
         "refresh_interval" : "1s"
      }
   }
}
client.indices.put_settings(index = index_name, body = index_settings)
settings = client.indices.get_settings(index = index_name)
pp.pprint(settings)

print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
client.indices.put_mapping(
    index=index_name,
    body={
        "properties": {
            "new_field": {"type": "text"}
        }
    }
)


mappings = client.indices.get_mapping(index = index_name)
pp.pprint(mappings)

print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
print(client.count(index = index_name))

## Add recipes to index with the correct mappings

In [7]:
for recipe_id in recipes_data:
    prep_time = recipes_data[recipe_id]['prepTimeMinutes']
    cook_time = recipes_data[recipe_id]['cookTimeMinutes']

    if prep_time is not None and cook_time is not None:
        time = prep_time + cook_time
    else:
        time = None
    
    recipe = {
        "recipe_id": recipe_id,
        "title": recipes_data[recipe_id]['displayName'],
        "title_embedding": None, # should call embedding function
        "description": recipes_data[recipe_id]['description'],
        "ingredients": recipes_data[recipe_id]['ingredients'],
        "time": time,
    }
    # Test first recipe log
    if recipe_id == '0': 
        print(recipe)

    ## Add recipe to index (not sure if it has the right mappings yet, so stay commented)
    # client.index(index=index_name, id=1, body=doc)
    

{'recipe_id': '0', 'title': 'How To Make Chicken Parmesan', 'title_embedding': None, 'description': 'Master the classic dish of chicken Parmesan by starting with the chicken, choosing a marinara sauce you love, and using a trio of cheese. ', 'ingredients': [{'displayText': '½ teaspoon of red pepper flakes (optional)', 'staple': False, 'ingredient': 'red pepper flakes', 'ingredientId': 'nx_item_00780', 'quantity': 0.5, 'unit': 'TEASPOON', 'preparation': None, 'brand': None, 'asinOverride': None, 'images': [], 'componentIndex': 0, 'productOverride': None}, {'displayText': '¼ cup of freshly grated Parmesan cheese', 'staple': False, 'ingredient': 'parmesan cheese', 'ingredientId': 'type_01033_00', 'quantity': 0.25, 'unit': 'CUP', 'preparation': None, 'brand': None, 'asinOverride': None, 'images': [], 'componentIndex': 0, 'productOverride': None}, {'displayText': '4 ounces of part-skim mozzarella, sliced', 'staple': False, 'ingredient': 'Cheese, mozzarella, part skim milk', 'ingredientId': 